<a href="https://colab.research.google.com/github/kiyong21c/chobocoding/blob/master/20220211_cv2_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# KNN 불일치 사항 재학습

In [10]:
from google.colab.patches import cv2_imshow # cv2.imshow()는 코랩에서 작동 안함
import cv2
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/digits.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_resize = cv2.resize(gray, (2000, 1000))

cells = [np.hsplit(row, 100) for row in np.vsplit(gray_resize, 50)] # [[0, ..., 99], [2row], [3row], ..., [50row]]

img = np.array(cells) # 리스트 -> 2차월 배열(array) : [[], [], [], ..., []]


# img.shape # (50, 100, 20, 20) : 50 X 100 배열, 배열의 각 요소는 20 X 20
# img.size # 2,000,000 : 50 X 100 X (20 X 20)
# img[0, 0] # 배열의 첫번째 요소(좌상단)도 역시 20X20의 2차월 배열[[0, ..., 19], [2row], [3row], ..., [20row]]

train = img[:,:50] # 50X100 X(20X20) 크기의 이미지를 가로 앞쪽 50크기로 가져옴
train.shape # (50, 50, 20, 20)
# 이러한 배열은 cv2_imshow() 불가, 2 or 3차원 배열이 아니기 때문

# train.reshape(-1, 20).shape # (50000, 20) 
# cv2_imshow(train.reshape(-1, 20)) # 글자하나의 가로폭크기로 전부 세로로 배치
# reshape(세로:유동적, 가로:20) 후 cv2_imshow() 가능 # 참고사이트 : https://rfriend.tistory.com/345

# 왜 가로(d)를 400크기? : tran/test의 shape이 (N, d) 일때, train_labels가 (N, 1) # 참고사이트 : https://deep-learning-study.tistory.com/286
# 따라서, N값을 맞출것을 고려하여 d값 선정
# 총 5000/2의 글자들 중 글자하나(20X20)를 (1X400)으로 길게 늘어뜨리고, 이에 맞는 세로(N)를 유동적(-1)으로 갖게함(N=2500)
train = train.reshape(-1, 400).astype(np.float32) # shape (N=2500, d=400)
test = img[:, 50:].reshape(-1, 400).astype(np.float32)


# 0 부터 9까지 라벨링 / train, test 라벨 생성
k = np.arange(10)
train_labels = np.repeat(k,250) # array([0, 0, 0, ..., 9, 9, 9]) : k의 각요소를 250번씩 반복
# 왜 250번이냐? : train/test를 나눴기 때문에 각 숫자당 250번씩만 씀
# 따라서, 0-9까지 10개의 숫자를 250번씩 할당하므로 N=2500 -> (2500, 1)
# 따라서, train/test의 shape도 (N, d)로 맞추어야 KNN 알고리즘의 knn.train() 적용가능

train_labels = train_labels[:, np.newaxis] # shape (N=2500, d=1)
test_labels = train_labels.copy()

# KNN 알고리즘 객체 생성
knn = cv2.ml.KNearest_create()
# KNN 알고리즘 학습
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
# (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
ret, result, neighbors, dist = knn.findNearest(test, k=5)

result # shape (2500, 1), # size 2500
# k값으로 이웃값을 찾은 결과 배열

matches = result == test_labels # 일치하면 True
mismatches = result != test_labels
correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
print('일치하는 nrows :', correct)

accuracy = correct*100.0/result.size
print(accuracy) # 91.76 : 인식 성공율

np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

with np.load('knn_data.npz') as data: # with로 열면 .close() 필요없음
    print(data.files) # ['train', 'train_labels']
    train = data['train']
    test = data['test']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # 불일치한(False) test 배열을 train 배열에 추가 : mismatches를 for문을 통해서 추가해야할듯!!!
    # mismatches를 한덩어리로 reshape하면 안되네
    for mismatch in mismatches: 
        train_add = test[np.where(mismatch), :].reshape(-1, 400) # train에 추가할 데이터(불일치한 test 배열)
        train_labels_add = train_labels[np.where(mismatch), :].reshape(-1, 1) # train_labels에 추가할 데이터(불일치한 test배열의 인덱스에 해당하는 train_labels)

        test_add = test[np.where(mismatch), :].reshape(-1,400)
        test_labels_add = test_labels[np.where(mismatch), :].reshape(-1, 1)
        # print(train_add.shape)
        # print(train_labels_add.shape)

        train = np.append(train, train_add, axis=0)
        train_labels = np.append(train_labels, train_labels_add, axis=0)

        test = np.append(test, test_add, axis=0)
        test_labels = np.append(test_labels, test_labels_add, axis=0)

    print('train.shape :', train.shape) # (2912, 400) 기존데이터 2500rows에 불일치했던 test 배열의 412rows를 append
    print('train_labels.shape :', train_labels.shape) # (2912, 1) 2500 + 412 = 2912
    print('test.shape :', test.shape)
    print('test_labels.shape :', test_labels.shape)

    # KNN 알고리즘 객체 생성
    knn = cv2.ml.KNearest_create()
    # KNN 알고리즘 학습
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
    # (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
    ret, result, neighbors, dist = knn.findNearest(test, k=5)

    result # shape (2500, 1), # size 2500
    # k값으로 이웃값을 찾은 결과 배열

    matches = result == test_labels # 일치하면 True
    mismatches = result != test_labels
    correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
    print('일치하는 nrows :', correct)

    accuracy = correct*100.0/result.size
    print(accuracy) # 91.76 : 인식 성공율

일치하는 nrows : 2294
91.76
['train', 'train_labels', 'test', 'test_labels']
train.shape : (2706, 400)
train_labels.shape : (2706, 1)
test.shape : (2706, 400)
test_labels.shape : (2706, 1)
일치하는 nrows : 2500
92.38728750923873


In [11]:
from google.colab.patches import cv2_imshow # cv2.imshow()는 코랩에서 작동 안함
import cv2
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/digits.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_resize = cv2.resize(gray, (2000, 1000))

cells = [np.hsplit(row, 100) for row in np.vsplit(gray_resize, 50)] # [[0, ..., 99], [2row], [3row], ..., [50row]]

img = np.array(cells) # 리스트 -> 2차월 배열(array) : [[], [], [], ..., []]


# img.shape # (50, 100, 20, 20) : 50 X 100 배열, 배열의 각 요소는 20 X 20
# img.size # 2,000,000 : 50 X 100 X (20 X 20)
# img[0, 0] # 배열의 첫번째 요소(좌상단)도 역시 20X20의 2차월 배열[[0, ..., 19], [2row], [3row], ..., [20row]]

train = img[:,:50] # 50X100 X(20X20) 크기의 이미지를 가로 앞쪽 50크기로 가져옴
train.shape # (50, 50, 20, 20)
# 이러한 배열은 cv2_imshow() 불가, 2 or 3차원 배열이 아니기 때문

# train.reshape(-1, 20).shape # (50000, 20) 
# cv2_imshow(train.reshape(-1, 20)) # 글자하나의 가로폭크기로 전부 세로로 배치
# reshape(세로:유동적, 가로:20) 후 cv2_imshow() 가능 # 참고사이트 : https://rfriend.tistory.com/345

# 왜 가로(d)를 400크기? : tran/test의 shape이 (N, d) 일때, train_labels가 (N, 1) # 참고사이트 : https://deep-learning-study.tistory.com/286
# 따라서, N값을 맞출것을 고려하여 d값 선정
# 총 5000/2의 글자들 중 글자하나(20X20)를 (1X400)으로 길게 늘어뜨리고, 이에 맞는 세로(N)를 유동적(-1)으로 갖게함(N=2500)
train = train.reshape(-1, 400).astype(np.float32) # shape (N=2500, d=400)
test = img[:, 50:].reshape(-1, 400).astype(np.float32)


# 0 부터 9까지 라벨링 / train, test 라벨 생성
k = np.arange(10)
train_labels = np.repeat(k,250) # array([0, 0, 0, ..., 9, 9, 9]) : k의 각요소를 250번씩 반복
# 왜 250번이냐? : train/test를 나눴기 때문에 각 숫자당 250번씩만 씀
# 따라서, 0-9까지 10개의 숫자를 250번씩 할당하므로 N=2500 -> (2500, 1)
# 따라서, train/test의 shape도 (N, d)로 맞추어야 KNN 알고리즘의 knn.train() 적용가능

train_labels = train_labels[:, np.newaxis] # shape (N=2500, d=1)
test_labels = train_labels.copy()

# KNN 알고리즘 객체 생성
knn = cv2.ml.KNearest_create()
# KNN 알고리즘 학습
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
# (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
ret, result, neighbors, dist = knn.findNearest(test, k=5)

result # shape (2500, 1), # size 2500
# k값으로 이웃값을 찾은 결과 배열

matches = result == test_labels # 일치하면 True
mismatches = result != test_labels
correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
print('일치하는 nrows :', correct)

accuracy = correct*100.0/result.size
print(accuracy) # 91.76 : 인식 성공율

np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

with np.load('knn_data.npz') as data: # with로 열면 .close() 필요없음
    print(data.files) # ['train', 'train_labels']
    train = data['train']
    test = data['test']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # 불일치한(False) test 배열을 train 배열에 추가 : mismatches를 for문을 통해서 추가해야할듯!!!
    # mismatches를 한덩어리로 해서 안됬나???
    for mismatch in mismatches: 
        train_add = test[np.where(mismatch), :].reshape(-1, 400) # train에 추가할 데이터(불일치한 test 배열)
        train_labels_add = train_labels[np.where(mismatch), :].reshape(-1, 1) # train_labels에 추가할 데이터(불일치한 test배열의 인덱스에 해당하는 train_labels)

        test_add = test[np.where(mismatch), :].reshape(-1,400)
        test_labels_add = test_labels[np.where(mismatch), :].reshape(-1, 1)
        # print(train_add.shape)
        # print(train_labels_add.shape)

        train = np.append(train, train_add, axis=0)
        train_labels = np.append(train_labels, train_labels_add, axis=0)

        test = np.append(test, test_add, axis=0)
        test_labels = np.append(test_labels, test_labels_add, axis=0)

    print('train.shape :', train.shape) # (2912, 400) 기존데이터 2500rows에 불일치했던 test 배열의 412rows를 append
    print('train_labels.shape :', train_labels.shape) # (2912, 1) 2500 + 412 = 2912
    print('test.shape :', test.shape)
    print('test_labels.shape :', test_labels.shape)

    # KNN 알고리즘 객체 생성
    knn = cv2.ml.KNearest_create()
    # KNN 알고리즘 학습
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
    # (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
    ret, result, neighbors, dist = knn.findNearest(test, k=5)

    result # shape (2500, 1), # size 2500
    # k값으로 이웃값을 찾은 결과 배열

    matches = result == test_labels # 일치하면 True
    mismatches = result != test_labels
    correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
    print('일치하는 nrows :', correct)

    accuracy = correct*100.0/result.size
    print(accuracy) # 91.76 : 인식 성공율

    np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

일치하는 nrows : 2294
91.76
['train', 'train_labels', 'test', 'test_labels']
train.shape : (2706, 400)
train_labels.shape : (2706, 1)
test.shape : (2706, 400)
test_labels.shape : (2706, 1)
일치하는 nrows : 2500
92.38728750923873


In [12]:
with np.load('knn_data.npz') as data: # with로 열면 .close() 필요없음
    print(data.files) # ['train', 'train_labels']
    train = data['train']
    test = data['test']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # 불일치한(False) test 배열을 train 배열에 추가 : mismatches를 for문을 통해서 추가해야할듯!!!
    # mismatches를 한덩어리로 해서 안됬나???
    for mismatch in mismatches: 
        train_add = test[np.where(mismatch), :].reshape(-1, 400) # train에 추가할 데이터(불일치한 test 배열)
        train_labels_add = train_labels[np.where(mismatch), :].reshape(-1, 1) # train_labels에 추가할 데이터(불일치한 test배열의 인덱스에 해당하는 train_labels)

        test_add = test[np.where(mismatch), :].reshape(-1,400)
        test_labels_add = test_labels[np.where(mismatch), :].reshape(-1, 1)
        # print(train_add.shape)
        # print(train_labels_add.shape)

        train = np.append(train, train_add, axis=0)
        train_labels = np.append(train_labels, train_labels_add, axis=0)

        test = np.append(test, test_add, axis=0)
        test_labels = np.append(test_labels, test_labels_add, axis=0)

    print('train.shape :', train.shape) # (2912, 400) 기존데이터 2500rows에 불일치했던 test 배열의 412rows를 append
    print('train_labels.shape :', train_labels.shape) # (2912, 1) 2500 + 412 = 2912
    print('test.shape :', test.shape)
    print('test_labels.shape :', test_labels.shape)

    # KNN 알고리즘 객체 생성
    knn = cv2.ml.KNearest_create()
    # KNN 알고리즘 학습
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
    # (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
    ret, result, neighbors, dist = knn.findNearest(test, k=5)

    result # shape (2500, 1), # size 2500
    # k값으로 이웃값을 찾은 결과 배열

    matches = result == test_labels # 일치하면 True
    mismatches = result != test_labels
    correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
    print('일치하는 nrows :', correct)

    accuracy = correct*100.0/result.size
    print(accuracy) # 91.76 : 인식 성공율

    np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

['train', 'train_labels', 'test', 'test_labels']
train.shape : (2912, 400)
train_labels.shape : (2912, 1)
test.shape : (2912, 400)
test_labels.shape : (2912, 1)
일치하는 nrows : 2706
92.92582417582418


In [13]:
with np.load('knn_data.npz') as data: # with로 열면 .close() 필요없음
    print(data.files) # ['train', 'train_labels']
    train = data['train']
    test = data['test']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # 불일치한(False) test 배열을 train 배열에 추가 : mismatches를 for문을 통해서 추가해야할듯!!!
    # mismatches를 한덩어리로 해서 안됬나???
    for mismatch in mismatches: 
        train_add = test[np.where(mismatch), :].reshape(-1, 400) # train에 추가할 데이터(불일치한 test 배열)
        train_labels_add = train_labels[np.where(mismatch), :].reshape(-1, 1) # train_labels에 추가할 데이터(불일치한 test배열의 인덱스에 해당하는 train_labels)

        test_add = test[np.where(mismatch), :].reshape(-1,400)
        test_labels_add = test_labels[np.where(mismatch), :].reshape(-1, 1)
        # print(train_add.shape)
        # print(train_labels_add.shape)

        train = np.append(train, train_add, axis=0)
        train_labels = np.append(train_labels, train_labels_add, axis=0)

        test = np.append(test, test_add, axis=0)
        test_labels = np.append(test_labels, test_labels_add, axis=0)

    print('train.shape :', train.shape) # (2912, 400) 기존데이터 2500rows에 불일치했던 test 배열의 412rows를 append
    print('train_labels.shape :', train_labels.shape) # (2912, 1) 2500 + 412 = 2912
    print('test.shape :', test.shape)
    print('test_labels.shape :', test_labels.shape)

    # KNN 알고리즘 객체 생성
    knn = cv2.ml.KNearest_create()
    # KNN 알고리즘 학습
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
    # (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
    ret, result, neighbors, dist = knn.findNearest(test, k=5)

    result # shape (2500, 1), # size 2500
    # k값으로 이웃값을 찾은 결과 배열

    matches = result == test_labels # 일치하면 True
    mismatches = result != test_labels
    correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
    print('일치하는 nrows :', correct)

    accuracy = correct*100.0/result.size
    print(accuracy) # 91.76 : 인식 성공율

    np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

['train', 'train_labels', 'test', 'test_labels']
train.shape : (3118, 400)
train_labels.shape : (3118, 1)
test.shape : (3118, 400)
test_labels.shape : (3118, 1)
일치하는 nrows : 2912
93.39320076972419


In [14]:
with np.load('knn_data.npz') as data: # with로 열면 .close() 필요없음
    print(data.files) # ['train', 'train_labels']
    train = data['train']
    test = data['test']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # 불일치한(False) test 배열을 train 배열에 추가 : mismatches를 for문을 통해서 추가해야할듯!!!
    # mismatches를 한덩어리로 해서 안됬나???
    for mismatch in mismatches: 
        train_add = test[np.where(mismatch), :].reshape(-1, 400) # train에 추가할 데이터(불일치한 test 배열)
        train_labels_add = train_labels[np.where(mismatch), :].reshape(-1, 1) # train_labels에 추가할 데이터(불일치한 test배열의 인덱스에 해당하는 train_labels)

        test_add = test[np.where(mismatch), :].reshape(-1,400)
        test_labels_add = test_labels[np.where(mismatch), :].reshape(-1, 1)
        # print(train_add.shape)
        # print(train_labels_add.shape)

        train = np.append(train, train_add, axis=0)
        train_labels = np.append(train_labels, train_labels_add, axis=0)

        test = np.append(test, test_add, axis=0)
        test_labels = np.append(test_labels, test_labels_add, axis=0)

    print('train.shape :', train.shape) # (2912, 400) 기존데이터 2500rows에 불일치했던 test 배열의 412rows를 append
    print('train_labels.shape :', train_labels.shape) # (2912, 1) 2500 + 412 = 2912
    print('test.shape :', test.shape)
    print('test_labels.shape :', test_labels.shape)

    # KNN 알고리즘 객체 생성
    knn = cv2.ml.KNearest_create()
    # KNN 알고리즘 학습
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
    # (2500, 400), layout: 학습 데이터 배치 방법, (2500, 1)
    ret, result, neighbors, dist = knn.findNearest(test, k=5)

    result # shape (2500, 1), # size 2500
    # k값으로 이웃값을 찾은 결과 배열

    matches = result == test_labels # 일치하면 True
    mismatches = result != test_labels
    correct = np.count_nonzero(matches) # 2294 : nonzero를 카운트 = True를 카운트
    print('일치하는 nrows :', correct)

    accuracy = correct*100.0/result.size
    print(accuracy) # 91.76 : 인식 성공율

    np.savez('knn_data.npz',train=train, train_labels=train_labels, test=test, test_labels=test_labels) # savez() : 여러배열을 저장, .npz 배열 저장 확장자

['train', 'train_labels', 'test', 'test_labels']
train.shape : (3324, 400)
train_labels.shape : (3324, 1)
test.shape : (3324, 400)
test_labels.shape : (3324, 1)
일치하는 nrows : 3118
93.80264741275572
